<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/CLASSIFICATION1_roberta_LoRA_stackexchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate
!pip install torch tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value
from collections import Counter

sof_dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/dataset/stackexchange_train_dataset_10000_title.csv',
    split='train'
)


# 결과를 확인
print(sof_dataset_train)

# 레이블의 데이터 건수 카운트
label_counter = Counter(example['label'] for example in sof_dataset_train)

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 10000
})
Counter({0: 2000, 1: 2000, 2: 2000, 3: 2000, 4: 2000})


In [ ]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value
from collections import Counter

sof_dataset_val = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/dataset/stackexchange_val_dataset_1000_title.csv',
    split='train'
)


# 결과를 확인
print(sof_dataset_val)

# 레이블의 데이터 건수 카운트
label_counter = Counter(example['label'] for example in sof_dataset_val)

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 1000
})
Counter({0: 200, 1: 200, 2: 200, 3: 200, 4: 200})


# Tokenizing

In [ ]:
from transformers import AutoTokenizer

sof_dataset= DatasetDict({
    "train": sof_dataset_train,
    "test": sof_dataset_val
})

# DatasetDict 출력
print(sof_dataset)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

def tokenize_function(examples):
    return tokenizer(examples["title_content"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = sof_dataset.map(tokenize_function, batched=True)


DatasetDict({
    train: Dataset({
        features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 1000
    })
})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# Model / LoRA config

In [ ]:
from peft import LoraConfig, TaskType
from peft import get_peft_model
from transformers import RobertaForSequenceClassification

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=8,lora_alpha=16, lora_dropout=0.1
)

model = RobertaForSequenceClassification.from_pretrained(
    'FacebookAI/roberta-large',
    num_labels=5
)

model = get_peft_model(model, lora_config)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Fine-tuning

In [ ]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics = {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"],
    }
    return metrics

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/LoRA/CLASSIFICATION1_roberta_LoRA_stackexchange",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=10)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.194900,0.186664,0.956000,0.957135,0.956000,0.955961
2,0.162800,0.127759,0.975000,0.975046,0.975000,0.974945
3,0.171400,0.141235,0.970000,0.970327,0.970000,0.969990
4,0.121100,0.144930,0.974000,0.974158,0.974000,0.974003
5,0.104700,0.143311,0.974000,0.974063,0.974000,0.973966
6,0.097100,0.139732,0.975000,0.975125,0.975000,0.974965
7,0.093300,0.138526,0.976000,0.976107,0.976000,0.975962
8,0.083600,0.131732,0.978000,0.978141,0.978000,0.977957
9,0.071400,0.129522,0.977000,0.977097,0.977000,0.976957
10,0.073300,0.135776,0.977000,0.977151,0.977000,0.976962


TrainOutput(global_step=12500, training_loss=0.1326159487915039, metrics={'train_runtime': 2473.1122, 'train_samples_per_second': 40.435, 'train_steps_per_second': 5.054, 'total_flos': 2.34399211008e+16, 'train_loss': 0.1326159487915039, 'epoch': 10.0})

# Inference

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer, device=0)

label_map = {
    0: "biology",
    1: "cooking",
    2: "diy",
    3: "travel",
    4: "stackoverflow"
}

# 주어진 텍스트를 토대로 예측을 5번 실행
texts = [
    "What are the theoretical and actual (measured) minimum water potentials in plants?",
    "What tastes like marigold? <p>I'm  looking to make Georgian Satsivi ",
    "How can I intentionally make my toilet make this noise? <p>For Halloween I want to haunt my toilets and have them make the noise heard here:",
    "What are the hours of operation of the Ankara Metro? <p>What are the times of the first and last trains on the Ankara Metro?</p>",
    "How to use \"HTML form target self\" ? I am new to HTML and need to complete a simple task.",
]

# 각 텍스트에 대해 예측 실행 및 레이블 출력
for i, text in enumerate(texts):
    prediction = classifier(text)
    predicted_label = prediction[0]['label']
    predicted_score = prediction[0]['score']

    # 레이블 이름을 가져옵니다 (여기서는 RoBERTa 모델의 레이블이 "LABEL_0", "LABEL_1" 등으로 되어있을 수 있으므로 매핑이 필요합니다).
    label_name = label_map[int(predicted_label.split('_')[-1])]
    print(f"Text {i+1}: Predicted label: {label_name}, Score: {predicted_score:.4f}")

The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification'

NameError: name 'infer' is not defined